In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import sys
import time
import lightgbm as lgb
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from IPython.display import HTML
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, hp, anneal, Trials
import hyperopt
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
usecols = ['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
        'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
        'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
        'canal_entrada', 'indfall','nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

# USECOLS ARE ALL THE COLUMNS THAT WE ARE USING FOR OUR PROJECT
# WE HAVE DROPPED A TOTAL OF 4 COLUMNS WHICH WERE TOO SKEWED.

usecols1 = ['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo','age', 
            'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel','indrel_1mes', 'tiprel_1mes',
        'indresi', 'indext','canal_entrada', 'indfall','nomprov', 'ind_actividad_cliente', 'renta', 'segmento']

# THESE ARE ALL THE FEATURE COLUMNS WE ARE USING.

usecols2 = ['fecha_dato','ncodpers','ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

# THESE ARE ALL THE PRODUCT COLUMNS .

# **DATA LOADING**

In [ ]:
# LOADING OF TRAIN AND TEST DATA IN df and df_t respectively.

df = pd.read_csv('../input/santander-pr/train.csv',dtype={"sexo":str,"ind_nuevo":str,"ult_fec_cli_1t":str,"indext":str,"ind_empleado":"category","ind_nuevo":"category","indrel":"category","indrel_1mes":"category","tiprel_1mes":"category","indresi":"category","indext":"category","conyuemp":"category","indfall":"category","canal_entrada":"category","ind_ahor_fin_ult1":"int8",'ind_aval_fin_ult1':"int8", 'ind_cco_fin_ult1':"int8",'ind_cder_fin_ult1':"int8", 'ind_cno_fin_ult1':"int8", 'ind_ctju_fin_ult1':"int8",'ind_ctma_fin_ult1':"int8", 'ind_ctop_fin_ult1':"int8", 'ind_ctpp_fin_ult1':"int8",'ind_deco_fin_ult1':"int8", 'ind_deme_fin_ult1':"int8", 'ind_dela_fin_ult1':"int8",'ind_ecue_fin_ult1':"int8", 'ind_fond_fin_ult1':"int8", 'ind_hip_fin_ult1':"int8",'ind_plan_fin_ult1':"int8", 'ind_pres_fin_ult1':"int8", 'ind_reca_fin_ult1':"int8",'ind_tjcr_fin_ult1':"int8", 'ind_valo_fin_ult1':"int8", 'ind_viv_fin_ult1':"int8", 'ind_recibo_ult1':"int8"}, usecols=usecols)
df_t = pd.read_csv('../input/santander-pr/test.csv',dtype={"sexo":str,"ind_nuevo":str,"ult_fec_cli_1t":str,"indext":str,"ind_empleado":"category","ind_nuevo":"category","indrel":"category","indrel_1mes":"category","tiprel_1mes":"category","indresi":"category","indext":"category","conyuemp":"category","indfall":"category","canal_entrada":"category"}, usecols=usecols1)

In [ ]:
# HERE WE ARE SORTING OUR DATA FOR USING LAGS AND TOGGLE FEATURES

df.sort_values(by=['ncodpers','fecha_dato'],inplace=True)

# **PRE-PROCESSING**

***The entire pre-processing is same as last notebook therefore we have put all of it in one cell***

In [ ]:
# WE Dropped around 27734 rows which had all its columns empty.

df=df[df['indrel'].notna()]

# Here we are converting our string dates into datetime datatype . 

#TRAIN
df["fecha_dato"] = pd.to_datetime(df["fecha_dato"],format="%Y-%m-%d")
df["fecha_alta"] = pd.to_datetime(df["fecha_alta"],format="%Y-%m-%d")
#TEST
df_t["fecha_dato"] = pd.to_datetime(df_t["fecha_dato"],format="%Y-%m-%d")
df_t["fecha_alta"] = pd.to_datetime(df_t["fecha_alta"],format="%Y-%m-%d")

# We are dropping antiguedad which refers to the seniority of a customer but rather we are using
# a new column cust_since which is the numeric difference between fecha_alta and fecha_dato since antiguedad had
# a lot of weird values.

# TRAIN
df.drop(['antiguedad'],axis=1,inplace=True)
df['cust_since'] = df['fecha_dato'].dt.to_period('M').astype(int) - df['fecha_alta'].dt.to_period('M').astype(int)
# TEST
df_t.drop(['antiguedad'],axis=1,inplace=True)
df_t['cust_since'] = df_t['fecha_dato'].dt.to_period('M').astype(int) - df_t['fecha_alta'].dt.to_period('M').astype(int)

# In column indrel_1mes the data was not uniform for similar data, some in integer some in string so we converted
# it using a dictionary.
map_dict = { 1.0  : "1",
            "1.0" : "1",
            "1"   : "1",
            "3.0" : "3",
            "P"   : "P",
            3.0   : "3",
            2.0   : "2",
            "3"   : "3",
            "2.0" : "2",
            4.0   : "4",
            "4.0" : "4",
            "4"   : "4",
            "2"   : "2"}

# TRAIN

df.indrel_1mes = df.indrel_1mes.apply(lambda x: map_dict.get(x,x))
# TEST

df_t.indrel_1mes = df_t.indrel_1mes.apply(lambda x: map_dict.get(x,x))


#We replaced the null values using 1

df.indrel_1mes.fillna("1",inplace=True)  # TRAIN
df_t.indrel_1mes.fillna("1",inplace=True) # TEST

# FILLING OF NULL VALUES

# WE HAVE FILLED THE NULL VALUES OF ALL COLUMNS USING THE MODE OF THE COLUMN.

# TRAIN

df['ind_empleado'].fillna("N",inplace=True)
df['pais_residencia'].fillna("ES",inplace=True)
df['indresi'].fillna("S",inplace=True)
df['indext'].fillna("N",inplace=True)
df['indfall'].fillna("N",inplace=True)
df['sexo'].fillna(df['sexo'].mode()[0], inplace=True)
df['segmento'].fillna(df['segmento'].mode()[0], inplace=True)
df['canal_entrada'] = df['canal_entrada'].astype('str')
df['canal_entrada'].fillna("UNK",inplace=True)
df.tiprel_1mes.fillna("A",inplace=True)
df['ind_nomina_ult1'].fillna(0,inplace=True)
df['ind_nom_pens_ult1'].fillna(0,inplace=True)
df.tiprel_1mes.replace(to_replace='N',value='A',inplace=True)

# TEST

df_t['ind_empleado'].fillna("N",inplace=True)
df_t['pais_residencia'].fillna("ES",inplace=True)
df_t['indresi'].fillna("S",inplace=True)
df_t['indext'].fillna("N",inplace=True)
df_t['indfall'].fillna("N",inplace=True)
df_t['sexo'].fillna(df_t['sexo'].mode()[0], inplace=True)
df_t['segmento'].fillna(df_t['segmento'].mode()[0], inplace=True)
df_t['canal_entrada'] = df_t['canal_entrada'].astype('str')
df_t['canal_entrada'].fillna("UNK",inplace=True)
df_t.tiprel_1mes.fillna("A",inplace=True)
df_t.tiprel_1mes.replace(to_replace='N',value='A',inplace=True)


#NOW WE WILL HANDLE THE NUMERICAL COLUMNS OF RENTA,AGE and CUST_SINCE.

# FIRST THE RENTA COLUMN.

# FOR REPLACING THE NULL VALUES OF RENTA WE GROUP IT WITH THEIR RESPECTIVE PROVINCE'S MEAN.
# AFTER THAT WE REMOVE OUTLIERS AT 75% QUANTILE MULTIPLIED BY 4 AND 25% QUANTILE DIVIDED BY 4.
# THEN WE STANDARDISE THE COLUMN.

# TRAIN

df['nomprov'].fillna(df['nomprov'].mode()[0], inplace=True)
df['renta']=df[['nomprov','renta']].groupby(['nomprov']).transform(lambda x: x.fillna(x.median())).renta
max_out=df.renta.quantile(0.75)*4
min_out=df.renta.quantile(0.25)/4
df.loc[df['renta']>max_out,'renta']=max_out
df.loc[df['renta']<min_out,'renta']=min_out
scale=StandardScaler().fit(df[['renta']])
df[['renta']]=scale.transform(df[['renta']])

# TEST 

df_t['nomprov'].fillna(df_t['nomprov'].mode()[0], inplace=True)
df_t['renta']=df_t[['nomprov','renta']].groupby(['nomprov']).transform(lambda x: x.fillna(x.median())).renta
max_out=df_t.renta.quantile(0.75)*4
min_out=df_t.renta.quantile(0.25)/4
df_t.loc[df_t['renta']>max_out,'renta']=max_out
df_t.loc[df_t['renta']<min_out,'renta']=min_out
scale=StandardScaler().fit(df_t[['renta']])
df_t[['renta']]=scale.transform(df_t[['renta']])


# SECOND THE AGE COLUMN
# WE REMOVE OUTLIERS OF AGES >95 and <12.
# THEN WE USE NORMALIZATION , MIN-MAX TECHNIQUE.

# TRAIN

df.age = pd.to_numeric(df.age,errors="coerce")
max_out=df.age.quantile(0.75)*1.9
min_out=df.age.quantile(0.25)/2
df.loc[df['age']>max_out,'age']=max_out
df.loc[df['age']<min_out,'age']=min_out
scale=MinMaxScaler().fit(df[['age']])
df[['age']]=scale.transform(df[['age']])

# TEST

df_t.age = pd.to_numeric(df_t.age,errors="coerce")
max_out=df_t.age.quantile(0.75)*1.9
min_out=df_t.age.quantile(0.25)/2
df_t.loc[df_t['age']>max_out,'age']=max_out
df_t.loc[df_t['age']<min_out,'age']=min_out
scale=MinMaxScaler().fit(df_t[['age']])
df_t[['age']]=scale.transform(df_t[['age']])


# THIRD THE CUST_SINCE COLUMN
# WE NORMALISE THE COLUMNS, MIN-MAX TECHNIQUE.

# TRAIN

scale=MinMaxScaler().fit(df[['cust_since']])
df[['cust_since']]=scale.transform(df[['cust_since']])

# TEST

scale=MinMaxScaler().fit(df_t[['cust_since']])
df_t[['cust_since']]=scale.transform(df_t[['cust_since']])


# NOW WE CHANGE DATA TYPES OF COLUMNS FOR BETTER RAM MANAGEMENT.

# TRAIN

df.ind_nuevo=df.ind_nuevo.astype(np.float64)
df.indrel=df.indrel.astype(np.float64)
df.ind_actividad_cliente=df.ind_actividad_cliente.astype(np.int8)
df.ind_nuevo=df.ind_nuevo.astype(np.int8)
df.indrel=df.indrel.astype(np.int8)
df.ind_nomina_ult1=df.ind_nomina_ult1.astype(np.int8)
df.ind_nom_pens_ult1=df.ind_nom_pens_ult1.astype(np.int8)
df.ind_empleado=df.ind_empleado.astype("category")
df.sexo=df.sexo.astype("category")

# TEST

df_t.ind_nuevo=df_t.ind_nuevo.astype(np.float64)
df_t.indrel=df_t.indrel.astype(np.float64)
df_t.ind_actividad_cliente=df_t.ind_actividad_cliente.astype(np.int8)
df_t.ind_nuevo=df_t.ind_nuevo.astype(np.int8)
df_t.indrel=df_t.indrel.astype(np.int8)
df_t.ind_empleado=df_t.ind_empleado.astype("category")
df_t.sexo=df_t.sexo.astype("category")


# IN MANY COLUMNS,WE HAD A LOT OF FEATURES AND IT WAS DIFFICULT TO ONE-HOT ENCODE THEM SO WE REDUCED THE NUMBER
# OF FEATURES IN COLUMN TO BE FINITE IN 2 COLUMNS.

# COLUMN CANAL_ENTRADA
# WE REDUCED THE NUMBER OF FEATURES TO BE THE 10 MOST POPULAR FEATURES AND ALL THE REST TO BE IN OTH.

canal_name={'KHE' : 'KHE',
            'KAT' : 'KAT',
            'KFC' : 'KFC',
            'KHQ' : 'KHQ',
            'KFA' : 'KFA',
            'KHK' : 'KHK',
            'KHM' : 'KHM',
            'UNK' : 'UNK',
            'KHD' : 'KHD',
            'KHN' : 'KHN'}
arr=list(df['canal_entrada'].unique())

for name in arr:
    if(name not in canal_name.keys()):
        canal_name[name]="OTH"

arr=list(df_t['canal_entrada'].unique()) 

for name in arr:
    if(name not in canal_name.keys()):
        canal_name[name]="OTH"

df.canal_entrada = df.canal_entrada.apply(lambda x: canal_name.get(x,x))  # TRAIN

df_t.canal_entrada = df_t.canal_entrada.apply(lambda x: canal_name.get(x,x))  # TEST


# COLUMN NOMPROV
# WE REDUCED THE NUMBER OF FEATURES TO BE THE 10 MOST POPULAR FEATURES AND ALL THE REST TO BE IN OTH.

city_name={'MADRID' : 'MADRID',
           'BARCELONA' : 'BARCELONA',
           'VALENCIA' : 'VALENCIA',
           'SEVILLA' : 'SEVILLA',
           'CORUÑA, A' : 'CORUNA',
           'MURCIA' : 'MURCIA',
           'MALAGA' : 'MALAGA',
           'ZARAGOZA' : 'ZARAGOZA',
           'ALICANTE' : 'ALICANTE',
           'CADIZ' : 'CADIZ'}
arr=list(df.nomprov.unique())
for name in arr:
    if(name not in city_name.keys()):
        city_name[name]="OTH"
        
arr=list(df_t.nomprov.unique())
for name in arr:
    if(name not in city_name.keys()):
        city_name[name]="OTH"
        
df.nomprov = df.nomprov.apply(lambda x: city_name.get(x,x)) # TRAIN

df_t.nomprov = df_t.nomprov.apply(lambda x: city_name.get(x,x)) # TEST

# **ENCODING AND MAKING OF DATAFRAMES**

In [ ]:
# MAKING OF df_train consisting of all feature columns from df dataframe. 

df_train = df.loc[:,'fecha_dato':'segmento']
df_train['cust_since']=df['cust_since']

# NOW WE DROP ANOTHER 6 COLUMNS BECAUSE THEY SHOWED NO IMPORTANCE IN OUR PLOT IMPORTANCE GRAPH.

drop=['ind_empleado','pais_residencia','indrel_1mes','indrel','indresi','indfall']
df_train.drop(columns=drop,inplace=True)
df_t.drop(columns=drop,inplace=True)

# NOW WE DO ONE-HOT ENCODING ON OUR FEATURE COLUMNS.

df_train=pd.get_dummies(df_train,columns=['nomprov','sexo','ind_nuevo','indext','canal_entrada','ind_actividad_cliente','segmento','tiprel_1mes'])

df_test=pd.get_dummies(df_t,columns=['nomprov','sexo','ind_nuevo','indext','canal_entrada','ind_actividad_cliente','segmento','tiprel_1mes'])

# df_y consists of all the target columns and few other important columns of which we find the lags and toggles.

df_y = df.loc[:,'ind_ahor_fin_ult1':'ind_recibo_ult1']
df_y['fecha_dato']=df['fecha_dato']
df_y['ncodpers']=df['ncodpers'] 
df_y['ind_actividad_cliente']=df['ind_actividad_cliente']
df_y['segmento']=df['segmento']


# NOW We have 3 dataframes df_train,df_test and df_y.


# **FEATURE ENGINEERING**

1. **PREPARING DATAFRAMES WHICH WE WILL MERGE WITH TEST DATA**
2. **STORING df_lag for all the lags that we will use ahead**
3. **Deleting df and df_t for RAM MANAGEMENT**

In [ ]:
df_cliente=df[['fecha_dato','ncodpers','ind_actividad_cliente']]
df_cliente = df_cliente[df_train['fecha_dato']=="2015-11-28"]

df_segmento=df[['fecha_dato','ncodpers','segmento']]
df_segmento = df_segmento[df_train['fecha_dato']=="2015-11-28"]

df_lag = df[usecols2]

del(df)
del(df_t)

**REDUCING TIMEFRAME TO ONLY THE REQUIRED TIMEFRAME**

In [ ]:
df_train = df_train[((df_train['fecha_dato']=="2015-10-28")| (df_train['fecha_dato']=="2016-04-28"))]
df_y = df_y[(df_y['fecha_dato']=="2015-10-28") | (df_y['fecha_dato']=="2016-04-28")]

**ADDING THE 2 NON-PRODUCT LAGS TO BOTH TEST AND TRAIN**

In [ ]:
# TRAIN

df_y.loc[df_y.ncodpers==df_y.ncodpers.shift(1),'ind_actividad_cliente'+'_lag']=df_y['ind_actividad_cliente'].shift(1)
df_y.loc[df_y.ncodpers==df_y.ncodpers.shift(1),'segmento'+'_lag']=df_y['segmento'].shift(1)

In [ ]:
# RENAMING BEFORE MERGE

df_cliente.rename(columns={'ind_actividad_cliente': 'ind_actividad_cliente' + '_lag'},inplace=True)
df_segmento.rename(columns={'segmento': 'segmento' + '_lag'},inplace=True)

# TEST

df_test=df_test.merge(df_cliente.loc[:, 'ncodpers':], on='ncodpers', how='left')
df_test=df_test.merge(df_segmento.loc[:, 'ncodpers':], on='ncodpers', how='left')

**REDUCING TIMELINE TO JUST TRAINING MONTH**

In [ ]:
df_train = df_train[((df_train['fecha_dato']=="2016-04-28"))]
df_y = df_y[(df_y['fecha_dato']=="2016-04-28")]

**FORMING DATAFRAME FOR LAGS**

**STORING 13 DATAFRAMES (12 EACH FOR TRAIN AND TEST)**

In [ ]:
# STORING LAGS FOR 12 MONTHS

df1 = df_lag[df_lag.fecha_dato == "2016-04-28"]

df2 = df_lag[df_lag.fecha_dato == "2016-03-28"]

df3 = df_lag[df_lag.fecha_dato == "2016-02-28"]

df4 = df_lag[df_lag.fecha_dato == "2016-01-28"]

df5 = df_lag[df_lag.fecha_dato == "2015-12-28"]

df6 = df_lag[df_lag.fecha_dato == "2015-11-28"]

df7 = df_lag[df_lag.fecha_dato == "2015-10-28"]

df8 = df_lag[df_lag.fecha_dato == "2015-09-28"]

df9 = df_lag[df_lag.fecha_dato == "2015-08-28"]

df10 = df_lag[df_lag.fecha_dato == "2015-07-28"]

df11 = df_lag[df_lag.fecha_dato == "2015-06-28"]

df12 = df_lag[df_lag.fecha_dato == "2015-05-28"]

df13 = df_lag[df_lag.fecha_dato == "2015-04-28"]

**MERGING OF LAG DATAFRAMES TO TRAIN DATA**

**FIRST RENAMING BEFORE MERGE**

In [ ]:
for c in df2.columns[2:]:
    df2.rename(columns={c:c+"_lag1"}, inplace=True)
    df3.rename(columns={c:c+"_lag2"}, inplace=True)
    df4.rename(columns={c:c+"_lag3"}, inplace=True)
    df5.rename(columns={c:c+"_lag4"}, inplace=True)
    df6.rename(columns={c:c+"_lag5"}, inplace=True)
    df7.rename(columns={c:c+"_lag6"}, inplace=True)
    df8.rename(columns={c:c+"_lag7"}, inplace=True)
    df9.rename(columns={c:c+"_lag8"}, inplace=True)
    df10.rename(columns={c:c+"_lag9"}, inplace=True)
    df11.rename(columns={c:c+"_lag10"}, inplace=True)
    df12.rename(columns={c:c+"_lag11"}, inplace=True)
    df13.rename(columns={c:c+"_lag12"}, inplace=True)

**MERGE TO TRAIN DATA**

In [ ]:
# MERGING 12 MONTH LAGS TO TRAIN DATA

df_y=df_y.merge(df2.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df3.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df4.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df5.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df6.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df7.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df8.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df9.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df10.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df11.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df12.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_y=df_y.merge(df13.loc[:, 'ncodpers':], on='ncodpers', how='left')


**MERGE OF LAG DATA FRAMES TO TEST DATA**

**RENAMING COLUMNS BEFORE MERGE**

In [ ]:
for c in df_lag.columns[2:]:
    df1.rename(columns={c:c+"_lag1"}, inplace=True)
    df2.rename(columns={c+"_lag1":c+"_lag2"}, inplace=True)
    df3.rename(columns={c+"_lag2":c+"_lag3"}, inplace=True)
    df4.rename(columns={c+"_lag3":c+"_lag4"}, inplace=True)
    df5.rename(columns={c+"_lag4":c+"_lag5"}, inplace=True)
    df6.rename(columns={c+"_lag5":c+"_lag6"}, inplace=True)
    df7.rename(columns={c+"_lag6":c+"_lag7"}, inplace=True)
    df8.rename(columns={c+"_lag7":c+"_lag8"}, inplace=True)
    df9.rename(columns={c+"_lag8":c+"_lag9"}, inplace=True)
    df10.rename(columns={c+"_lag9":c+"_lag10"}, inplace=True)
    df11.rename(columns={c+"_lag10":c+"_lag11"}, inplace=True)
    df12.rename(columns={c+"_lag11":c+"_lag12"}, inplace=True)

**MERGE TO TEST DATA**

In [ ]:
# MERGING 12 MONTHS LAG INTO TEST DATA

df_test=df_test.merge(df1.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df2.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df3.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df4.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df5.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df6.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df7.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df8.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df9.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df10.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df11.loc[:, 'ncodpers':], on='ncodpers', how='left')

df_test=df_test.merge(df12.loc[:, 'ncodpers':], on='ncodpers', how='left')


**DELETING ALL THE DATAFRAMES FOR BETTER RAM MANAGEMENT**

In [ ]:
del(df2)
del(df3)
del(df4)
del(df5)
del(df6)
del(df7)
del(df8)
del(df9)
del(df10)
del(df11)
del(df12)
del(df13)

**REMOVING NULL VALUES AND OTHER PROCESSING IN THE NEWLY CREATED COLUMNS**

In [ ]:
# TRAIN DATA

df_y.fillna(0,inplace=True)
df_y.loc[df_y.ind_actividad_cliente_lag.isnull()==True,'ind_actividad_cliente_lag']=df_y.ind_actividad_cliente
df_y.loc[df_y.segmento_lag.isnull()==True,'segmento_lag']=df_y.segmento

# TEST DATA

df_test.loc[(df_test.ind_actividad_cliente_lag.isnull()==True) & (df_test.ind_actividad_cliente_0==1),'ind_actividad_cliente_lag']=0
df_test.loc[(df_test.ind_actividad_cliente_lag.isnull()==True) & (df_test.ind_actividad_cliente_1==1),'ind_actividad_cliente_lag']=1
df_test.loc[(df_test.segmento_lag.isnull()==True) & (df_test['segmento_01 - TOP']==1) ,'segmento_lag']='TOP'
df_test.loc[(df_test.segmento_lag.isnull()==True) & (df_test['segmento_02 - PARTICULARES']==1) ,'segmento_lag']='PARTICULARES'
df_test.loc[(df_test.segmento_lag.isnull()==True) & (df_test['segmento_03 - UNIVERSITARIO']==1) ,'segmento_lag']='UNIVERSITARIO'
df_test.fillna(0,inplace=True)

# CHANGING DATA TYPES

#TRAIN

for c in df_y.columns[78:]:
    df_y[c]=df_y[c].astype("int8")

#TEST

for c in df_test.columns[44:379]:
    df_test[c]=df_test[c].astype("int8")

# CONVERTING STRING DATA INTO NUMERICALS.

convert={"02 - PARTICULARES" : 1,
         "PARTICULARES" : 1,
         "03 - UNIVERSITARIO" : 2,
         "UNIVERSITARIO" : 2,
         "01 - TOP" : 3,
         "TOP" : 3}

df_y.segmento_lag = df_y.segmento_lag.apply(lambda x: convert.get(x,x))

df_test.segmento_lag = df_test.segmento_lag.apply(lambda x: convert.get(x,x))

**SOME OTHER WORK BEFORE TRAINING**

In [ ]:
# SHIFTING OF ALL LAG COLUMNS FROM df_y to df_train and dropping them from df_y

mer=df_y[df_y.columns[25:]]

df_train=df_train.merge(mer,on='ncodpers',how='left')

for c in df_y.columns[26:]:
    df_y.drop(columns=c,inplace=True)
    
# REALIIGNING OF TRAIN AND TEST COLUMNS

col=df_test.columns
df_train=df_train[col]

# MAKING OF THE result dataframe.

result_curr=pd.DataFrame()
result_curr['ncodpers']=df_test['ncodpers']

# DROPPING OF COLUMNS WHICH ARE NOT NEEDED IN TRAINING PHASE.

df_train.drop(columns=['fecha_dato','ncodpers', 'fecha_alta'], inplace=True)
df_test.drop(columns=['fecha_dato','ncodpers', 'fecha_alta'],inplace=True)
df_y.drop(columns=['fecha_dato'], inplace=True)

# **TRAINING**

**MODEL SELECTION**

In [ ]:
clf = lgb.LGBMClassifier(          max_depth=11,
                                   n_estimators=700,
                                   learning_rate=0.026,
                                   num_leaves=105,
                                   min_child_weight=25,
                                   colsample_bytree=0.42,
                                   objective='binary',
                                   n_jobs=-1)

**TRAINING FOR OUR DATA**

In [ ]:
for c in df_y.columns[:-1]:
    y_train = df_y[c]
    clf.fit(df_train, y_train)
    p_train=clf.predict_proba(df_test)[:,1]
    result_curr[c]=p_train

# **ENSEMBLING**

**READING THE OTHER NOTEBOOK'S RESULT**

In [ ]:
result_old=pd.read_csv('../input/old-result/result_old.csv')

In [ ]:
result=pd.DataFrame()
result['ncodpers']=result_curr['ncodpers']

**WE WILL ENSEMBLE result_curr and result_old**

In [ ]:
for c in result_curr.columns[1:]:
    result[c]=((0.58*result_curr[c]) + (0.42*result_old[c]))

# **CONVERSION TO MAP5 FORMAT**

In [ ]:
result=result.merge(df1.loc[:, 'ncodpers':], on='ncodpers', how='left')
result.fillna(0,inplace=True)

In [ ]:
for c in result_curr.columns[1:]:
    result[c+'_ans']=abs(result[c]-result[c+'_lag1'])
finalresult=pd.DataFrame(columns=['ncodpers', 'changed'])
finalresult['ncodpers']=result['ncodpers']

In [ ]:
yes=[]
for i in range(len(result)):
    toadd=[]
    for j in result.columns[1:25]:
        toadd.append([result[j+"_ans"][i], j])
    toadd.sort(key=lambda x: -x[0])
    toadd=toadd[0:5]
    toadd1=""
    for j in toadd:
        toadd1=toadd1+j[1]+" "
    yes.append(toadd1)
finalresult['changed']=yes

In [ ]:
finalresult.to_csv('FinalResult.csv',index=False)